In [36]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI #before: langchain.chat_models
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
import os
from langchain.chains import SequentialChain
import time


load_dotenv()
handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [37]:
MODEL = "gemini-2.0-flash" # before: gpt-4-1106-preview
# ORGANISATION = "org-uZrowMmGKXM2eaDHBuh0Qfs4"

OUTPUT_FOLDER = "generated_guis_mm"
DATASET = "dataset.csv"

df = pd.read_csv(DATASET, sep=";")

In [38]:
from prompts import SUMMARY2REQUIREMENTS_ZS, REQUIREMENTS2IMPLEMENTATION_ZS, IMPLEMETATION2STRUCTURE_ZS, STRUCTURE2CODE_ZS

#PD_ZS
def pd_zs(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIREMENTS_ZS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_ZS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_ZS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))


In [39]:
from prompts import SUMMARY2REQUIRMENTS_FS, REQUIREMENTS2IMPLEMENTATION_FS, IMPLEMETATION2STRUCTURE_FS, STRUCTURE2CODE_FS

#PD_FS
def pd_fs(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIRMENTS_FS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_FS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_FS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_FS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [40]:
from prompts import SUMMARY2CODE_ZS

#Instruction
def instruction(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=SUMMARY2CODE_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [41]:
from prompts import REFINE_SUMMARY_ZS

def ref_summary(summary, temp=1):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=REFINE_SUMMARY_ZS, callbacks=[handler])

    try:
        response = chain.run(summary=summary, callbacks=[handler])

    except Exception as e:
        print(e)

    return(response)

In [42]:
from prompts import REF_SUMMARY2CODE_ZS

#Refined instruction
def ref_instruction(summary, temp=1):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=REF_SUMMARY2CODE_ZS, callbacks=[handler])

    try:
        refined_summary = ref_summary(summary)
        response = chain.run(summary=refined_summary, callbacks=[handler])
    except Exception as e:
        print(e)

    return(response)

In [43]:
import re

def extract_html(code, file, folder):
    html_content = re.search(r'<!DOCTYPE html>.*</html>', code, re.DOTALL)
    extracted_text = html_content.group()
    try:
        with open(f"{folder}/{file}.html", 'w') as f:
            f.write(extracted_text)
    except Exception as e:
        print(f"{file} failed")
        print(e)

In [44]:
def prototyping(summary, folder):
    timings = {}

    start = time.time()
    code = instruction(summary)
    extract_html(code, "instruction", folder)
    timings["instruction"] = time.time() - start

    start = time.time()
    code = pd_zs(summary)
    extract_html(code, "pd_zs", folder)
    timings["pd_zs"] = time.time() - start

    start = time.time()
    code = pd_fs(summary)
    extract_html(code, "pd_fs", folder)
    timings["pd_fs"] = time.time() - start

    start = time.time()
    code = ref_instruction(summary)
    extract_html(code, "ref_instruction", folder)
    timings["ref_instruction"] = time.time() - start

    return timings


In [54]:
OUTPUT_FOLDER = "generated_guis_mm"
DATASET = "rico_guis_business_descriptions_short"

df = pd.read_csv(DATASET, sep=",")

In [66]:
def pipeline():
    all_timings = {}
    for index, row in df.iterrows():
        # if row['UI Number'] in [3261, 38961, 53054, 69587]
        # if row['UI_Number'] in [30982, 18784, 18782, 67044, 67045, 63575, 59370, 54377]:
        # if row['UI_Number'] in [34346, 22151, 34527, 67033, 9015, 49799, 49794, 9007]:
        if row['UI_Number'] in [49794, 49799, 67033, 9007, 9015]:
            path = f"{OUTPUT_FOLDER}/{row['UI_Number']}"
            os.makedirs(path)
            timings = prototyping(row['Summary'], path)
            all_timings[row['UI_Number']] = timings
    return all_timings


In [67]:
pipeline()

AttributeError: 'NoneType' object has no attribute 'group'

{69587: {'instruction': 9.580247402191162,
         'pd_zs': 45.176225662231445,
         'pd_fs': 30.377336502075195,
         'ref_instruction': 33.990472078323364},
 3261: {'instruction': 12.721147060394287,
        'pd_zs': 54.26401472091675,
        'pd_fs': 31.441614866256714,
        'ref_instruction': 29.374341249465942},
 38961: {'instruction': 13.053589105606079,
         'pd_zs': 58.20743012428284,
         'pd_fs': 32.09258055686951,
         'ref_instruction': 33.643258571624756},
 53054: {'instruction': 10.029322147369385,
         'pd_zs': 54.366239070892334,
         'pd_fs': 32.76721167564392,
         'ref_instruction': 40.38598871231079}}

{18782: {'instruction': 11.885533332824707,
  'pd_zs': 57.84360647201538,
  'pd_fs': 26.193729400634766,
  'ref_instruction': 32.07602643966675},
 18784: {'instruction': 11.928243398666382,
  'pd_zs': 52.73217415809631,
  'pd_fs': 29.910811185836792,
  'ref_instruction': 29.38361644744873},
 30982: {'instruction': 9.59986686706543,
  'pd_zs': 76.6120491027832,
  'pd_fs': 34.17509913444519,
  'ref_instruction': 28.00716519355774},
 54377: {'instruction': 10.42833685874939,
  'pd_zs': 40.320576906204224,
  'pd_fs': 32.92193937301636,
  'ref_instruction': 23.70571208000183},
 59370: {'instruction': 9.471755981445312,
  'pd_zs': 49.06912803649902,
  'pd_fs': 25.46076512336731,
  'ref_instruction': 31.195977687835693},
 63575: {'instruction': 11.746174573898315,
  'pd_zs': 46.03599977493286,
  'pd_fs': 23.44306492805481,
  'ref_instruction': 41.296321392059326},
 67044: {'instruction': 9.348301410675049,
  'pd_zs': 57.41723370552063,
  'pd_fs': 34.19824838638306,
  'ref_instruction': 36.75452423095703},
 67045: {'instruction': 9.979282140731812,
  'pd_zs': 44.74830603599548,
  'pd_fs': 26.10989022254944,
  'ref_instruction': 21.338813304901123}}

In [20]:
def test_pipeline():
    for index, row in df.iterrows():
        code = ref_instruction(row['Summary'])
        file = "ref_instruction"
        folder = f"{OUTPUT_FOLDER}/{row['UI Number']}"

        html_content = re.search(r'<!DOCTYPE html>.*</html>', code, re.DOTALL)
        extracted_text = html_content.group()
        try:
            with open(f"{folder}/{file}.html", 'w') as f:
                f.write(extracted_text)
                print(row['UI Number'])
        except Exception as e:
            print(f"{file} failed")
            print(e)

test_pipeline()

39054


KeyboardInterrupt: 